# Subgoal sequences
This file analysis differences between different subgoal sequences.

In [ ]:
# set up imports
import os
import sys
__file__ = os.getcwd()
proj_dir =  os.path.dirname(os.path.realpath(__file__))
sys.path.append(proj_dir)
utils_dir = os.path.join(proj_dir,'utils')
sys.path.append(utils_dir)
analysis_dir = os.path.join(proj_dir,'analysis')
analysis_utils_dir = os.path.join(analysis_dir,'utils')
sys.path.append(analysis_utils_dir)
agent_dir = os.path.join(proj_dir,'model')
sys.path.append(agent_dir)
agent_util_dir = os.path.join(agent_dir,'utils')
sys.path.append(agent_util_dir)
experiments_dir = os.path.join(proj_dir,'experiments')
sys.path.append(experiments_dir)
df_dir = os.path.join(proj_dir,'results/dataframes')

In [ ]:
from model.Subgoal_Planning_Agent import *
import utils.blockworld as bw
import utils.blockworld_library as bl

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
import model.utils.decomposition_functions

In [ ]:
#inline plots
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [ ]:
plt.rcParams["figure.figsize"] = (20,7)
plt.rcParams.update({'font.size': 22})

In [ ]:
#display all columns
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 20)
pd.set_option('display.max_colwidth', 100)
pd.set_option('display.min_rows', 6)

In [ ]:
#helper function for pd.agg
def item(x):
    return x.tail(1).item()

Let's load the results of the experiment

In [ ]:
df_paths = ['subgoal planning full BFS2.pkl',
'subgoal planning full bogo.pkl',
'subgoal planning full BFS1.pkl']

In [ ]:
df_paths = ['subgoal planning full BFS0 to 2 small.pkl']

In [ ]:
df_paths = ['simulated lookaheads tiny.pkl']

In [ ]:
#load all experiments as one dataframe
df = pd.concat([pd.read_pickle(os.path.join(df_dir,l)) for l in df_paths])
print("Loaded dataframe")

In [ ]:
#alternatively, choose csv
df_paths = ['simulated lookaheads tiny.csv']

In [ ]:
#load all experiments as one dataframe from CSV
df = pd.concat([pd.read_csv(os.path.join(df_dir,l)) for l in df_paths])
print("Loaded dataframe")

Let's choose just one agent to make it easier to interpret

In [ ]:
oneadf = df[df['parent: lower level: horizon'] == 2]

In [ ]:
oneadf

creating `fdf` with only outcomes

In [ ]:
fdf = oneadf.groupby('run_ID').agg({
    'agent_label' : item,
    'world' : item,
    'c_weight' : item,
    'sequence_length' : item,
    'include_subsequences' : item,
    'parent: lower level: agent_type' : item,
    'parent: lower level: scoring_function' : item,
    'parent: lower level: horizon' : item,
    'partial_planning_cost':['sum','mean',np.std],
    'partial_solution_cost':['sum','mean',np.std],
    'planning_cost':['sum','mean',np.std],
    'solution_cost':['sum','mean',np.std],
    'all_sequences_planning_cost':['sum','mean',np.std], #includes penalty and therefore is meaningless
    'world_status' : lambda x:x.tail(1).item(),
    'decomposed_silhouette' : 'count' #how many subgoals did we act out? With stepsize of 1 number of subgoals chosen
})

#flatten the dataframe to remove multi-index for next groupby
fdf.columns = [' '.join(col).strip() for col in fdf.columns.values]
fdf.reset_index(inplace=True)

In [ ]:
fdf

Let's look at the effect of sequence length

In [ ]:
length_df = fdf.groupby('sequence_length item').agg({
    'world_status <lambda>' : lambda x:len([r for r in x if r == 'Win'])/len(x),
     'partial_planning_cost sum':'mean',
     'partial_solution_cost sum':'mean',
     'partial_solution_cost mean':'mean',
     'planning_cost sum':'mean',
     'planning_cost mean':'mean',
#      'solution_cost':'mean', #includes penalty and therefore is meaningless
     'all_sequences_planning_cost sum':'mean',
     'all_sequences_planning_cost mean':'mean',
     'decomposed_silhouette count' : 'mean'
})

In [ ]:
length_df

In [ ]:
plt.bar(length_df.index,length_df['world_status <lambda>'])
plt.title("Proportion perfect reconstruction")
plt.ylabel("Proportion perfect reconstruction")
plt.xlabel("Sequence length")
plt.show()

plt.bar(length_df.index,length_df['all_sequences_planning_cost sum'])
plt.title("Mean sum total planning cost over all sequences")
plt.ylabel("States evaluated")
plt.xlabel("Sequence length")
plt.show()

plt.bar(length_df.index,length_df['all_sequences_planning_cost mean'])
plt.title("Mean mean total planning cost over all sequences")
plt.ylabel("States evaluated")
plt.xlabel("Sequence length")
plt.show()

plt.bar(length_df.index,length_df['planning_cost sum'])
plt.title("Mean sum of planning costs for chosen sequence")
plt.ylabel("States evaluated")
plt.xlabel("Sequence length")
plt.show()

plt.bar(length_df.index,length_df['planning_cost mean'])
plt.title("Mean mean of planning costs for chosen sequence")
plt.ylabel("States evaluated")
plt.xlabel("Sequence length")
plt.show()

plt.bar(length_df.index,length_df['partial_solution_cost mean'])
plt.title("Mean solution cost")
plt.ylabel("States evaluated")
plt.xlabel("Sequence length")
plt.show()

plt.bar(length_df.index,length_df['decomposed_silhouette count'])
plt.title("Mean number of subgoals")
plt.ylabel("Number of subgoals acted out")
plt.xlabel("Sequence length")
plt.show()

plt.bar(length_df.index,length_df['partial_solution_cost mean'] / (1/length_df['decomposed_silhouette count']))
plt.title("Solution cost normalized by number of subgoals acted out")
plt.ylabel("Cost / (1/# subgoals)")
plt.xlabel("Sequence length")
plt.show()

Let's look at c_weight

In [ ]:
cw_df = fdf.groupby('c_weight item').agg({
    'world_status <lambda>' : lambda x:len([r for r in x if r == 'Win'])/len(x),
     'partial_planning_cost sum':'mean',
     'partial_solution_cost sum':'mean',
     'partial_solution_cost mean':'mean',
     'planning_cost sum':'mean',
     'planning_cost mean':'mean',
#      'solution_cost':'mean', #includes penalty and therefore is meaningless
     'all_sequences_planning_cost sum':'mean',
         'all_sequences_planning_cost mean':'mean',
    'decomposed_silhouette count' : 'mean'
})

In [ ]:
cw_df

In [ ]:
plt.plot(cw_df.index,cw_df['world_status <lambda>'])
plt.title("Proportion perfect reconstruction")
plt.ylabel("Proportion perfect reconstruction")
plt.xlabel("$\lambda$")
plt.show()

plt.plot(cw_df.index,cw_df['all_sequences_planning_cost sum'])
plt.title("Mean sum total planning cost over all sequences")
plt.ylabel("States evaluated")
plt.xlabel("$\lambda$")
plt.show()

plt.plot(cw_df.index,cw_df['all_sequences_planning_cost mean'])
plt.title("Mean mean total planning cost over all sequences")
plt.ylabel("States evaluated")
plt.xlabel("$\lambda$")
plt.show()

plt.plot(cw_df.index,cw_df['partial_planning_cost sum'])
plt.title("Mean sum of partial planning costs for chosen sequence")
plt.ylabel("States evaluated")
plt.xlabel("$\lambda$")
plt.show()

plt.plot(cw_df.index,cw_df['planning_cost sum'])
plt.title("Mean sum of planning costs for chosen sequence")
plt.ylabel("States evaluated")
plt.xlabel("$\lambda$")
plt.show()

plt.plot(cw_df.index,cw_df['partial_solution_cost mean'])
plt.title("Mean solution cost")
plt.ylabel("States evaluated")
plt.xlabel("$\lambda$")
plt.show()

plt.plot(cw_df.index,cw_df['decomposed_silhouette count'])
plt.title("Mean number of subgoals")
plt.ylabel("Number of subgoals acted out")
plt.xlabel("$\lambda$")
plt.show()

plt.plot(cw_df.index,cw_df['partial_solution_cost mean'] / (1/cw_df['decomposed_silhouette count']))
plt.title("Solution cost normalized by number of subgoals acted out")
plt.ylabel("Cost / (1/# subgoals)")
plt.xlabel("$\lambda$")
plt.show()



lets group by agent and the make a big scatter plot :)

In [ ]:
a_df = fdf.groupby('agent_label item').agg({
    'world_status <lambda>' : lambda x:len([r for r in x if r == 'Win'])/len(x),
     'partial_planning_cost sum':'mean',
     'partial_solution_cost sum':'mean',
     'partial_solution_cost mean':'mean',
     'planning_cost sum':'mean',
     'planning_cost mean':'mean',
#      'solution_cost':'mean', #includes penalty and therefore is meaningless
     'all_sequences_planning_cost sum':'mean',
    'decomposed_silhouette count' : 'mean',
    'c_weight item' : item
})

In [ ]:
plt.scatter(a_df['all_sequences_planning_cost sum'],a_df['partial_solution_cost sum'],c = a_df['c_weight item'])
plt.title("Total planning cost vs partial solution cost")
plt.colorbar(label="$\lambda$")
plt.xlabel("Total planninng cost")
plt.ylabel("Partial solution cost")

In [ ]:
plt.scatter(a_df['decomposed_silhouette count'],a_df['planning_cost sum'],c = a_df['c_weight item'])
plt.title("Number of actual subgoals vs planning cost")
plt.colorbar(label="$\lambda$")
plt.xlabel("Number of actual subgoals")
plt.ylabel("Number of states evaluated")

In [ ]:
plt.scatter(a_df['decomposed_silhouette count'],a_df['all_sequences_planning_cost sum'],c = a_df['c_weight item'])
plt.title("Number of actual subgoals vs Total planning cost")
plt.colorbar(label="$\lambda$")
plt.xlabel("Number of actual subgoals")
plt.ylabel("Total planning cost")

In [ ]:
plt.scatter(a_df['decomposed_silhouette count'],a_df['partial_solution_cost mean'],c = a_df['c_weight item'])
plt.title("Number of actual subgoals vs partial solution cost")
plt.colorbar(label="$\lambda$")
plt.xlabel("Number of actual subgoals")
plt.ylabel("partial solution cost")

## Sandgraph! 🏝

Since loading the pickled data takes too long, we sketchily recreate the names from the string of the decomposition

In [ ]:
import re
import ast
def str2array(s):
    #strip "array" and parentheses
    s=re.sub('\[array\(', '', s.strip())
    s=re.sub('\)]', '', s.strip())
    # Remove space after [
    s=re.sub('\[ +', '[', s.strip())
    # Replace commas and spaces
    s=re.sub('[,\s]+', ', ', s)
    return np.array(ast.literal_eval(s))

In [ ]:
names = []
for run in oneadf.groupby('run_ID'):
    silhouettes = run[1]['decomposed_silhouette'].dropna()
    _names = [np.sum(str2array(s).sum(axis=1) != 0) for s in silhouettes]
    names.append(_names)

In [ ]:
from matplotlib.patches import Rectangle
from matplotlib import cm

In [ ]:
plt.plot()
colors = ["red","orange","yellow","green","blue","purple","grey","black","pink"]
for x,n in enumerate(names):
    for i,g in enumerate(reversed(n)):
        plt.gca().add_patch(Rectangle((x,0),1,g, facecolor = colors[len(n) - i]))